Set up imports/paths

In [5]:
import numpy as np
import keras
from datetime import datetime
from training_functions import *
data_dir = "Insert Correct Data Directory" #TODO
requests_file = 'request_yamls/training.yaml'
saved_json = 'training.json'

Break up data into training/test

In [ ]:
### Only run this cell if there is no training.json file already saved ###
save_training_json(requests_file, data_dir, saved_json)

In [ ]:
with open(saved_json) as json_file:
    data = ujson.load(json_file)

# Uncomment end of following line to train with both hands, currently only using left-hand data
corresponding_streams = [('myo-left/emg', 'tactile-glove-left/tactile_data')] #,('myo-right/emg', 'tactile-glove-right/tactile_data')]
window_time = 2 # in seconds
train_x, train_y, test_x, test_y = window_data(data, corresponding_streams, window_time)


In [ ]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

In [9]:
# replace np.mean with other averaging functions in parse_hdf5_data.py as complexity increases
# currently just averaging all numbers in 32x32 grid and 1x8 emg array
train_x = np.mean(train_x, tuple(range(2, train_x.ndim)), keepdims=True)
train_y = np.mean(train_y, tuple(range(2, train_y.ndim)), keepdims=True)
test_x = np.mean(test_x, tuple(range(2, test_x.ndim)), keepdims=True)
test_y = np.mean(test_y, tuple(range(2, test_y.ndim)), keepdims=True)

In [ ]:
print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

Create Model

In [ ]:
model = setup_model(train_x)

Training/Plotting loop

In [ ]:
# https://keras.io/guides/training_with_built_in_methods/
log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=train_x, 
          y=train_y, 
          batch_size=10,
          epochs=10, 
          validation_data=(test_x, test_y), 
          callbacks=[tensorboard_callback])

Evaluate trained model

In [20]:
# Can uncomment this once data is split into training, validation, AND testing
# Currently only being split into training and test, with "test" being used to validate in the previous cell

# results = model.evaluate(test_x, test_y, batch_size=128)
# print("test loss, test acc:", results)